In this example we solve a fitted parabolic PDE  with a discontinuous Galerkin variational time discretization.

## Imports

In [ ]:
from netgen.geom2d import unit_square
from ngsolve import *
from xfem import *
from math import pi
ngsglobals.msg_level = 1

## basic parameters

In [ ]:
# Space finite element order
order = 4
# Time finite element order
k_t = 2
# Final simulation time
tend = 1.0
# Time step
delta_t = 1 / 4

## background mesh

In [ ]:
from ngsolve.meshes import *
mesh = Make1DMesh(5000)

## Space-Time FESpace

In [ ]:
V = H1(mesh, order=order, dirichlet=".*")
tfe = ScalarTimeFE(k_t)
st_fes = tfe * V

## Data

In [ ]:
tnew = 0
told = Parameter(0)
t = told + delta_t * tref

u_exact = sin(pi * t) * sin(pi * x)

coeff_f = u_exact.Diff(t) - u_exact.Diff(x).Diff(x)
coeff_f = coeff_f.Compile()


## Symbols for the definition of the discrete variational formulation

In [ ]:
gfu = GridFunction(st_fes)
u_last = CreateTimeRestrictedGF(gfu, 1)
u, v = st_fes.TnT()

dxt = delta_t * dxtref(mesh, time_order=2*k_t+2)
dxold = dmesh(mesh, tref=0)
dxnew = dmesh(mesh, tref=1)
def dt(u):
    return 1.0 / delta_t * dtref(u)

## the discrete variational formulation

In [ ]:
a = BilinearForm(st_fes, symmetric=False)
a += grad(u) * grad(v) * dxt
a += u * v * dxold
a += dt(u) * v * dxt
a.Assemble()
ainv = a.mat.Inverse(st_fes.FreeDofs(), "umfpack")

f = LinearForm(st_fes)
f += coeff_f * v * dxt
f += u_last * v * dxold

## Setup and solution of linear systems

In [ ]:
u_last.Set(fix_tref(u_exact, 0))
Draw(u_last, mesh, "u")

while tend - told.Get() > delta_t / 2:
    f.Assemble()
    gfu.vec.data = ainv * f.vec
    RestrictGFInTime(spacetime_gf=gfu, reference_time=1.0, space_gf=u_last)
    l2error = sqrt(Integrate((u_exact - gfu)**2 * dxnew, mesh))
    Redraw()
    told.Set(told.Get() + delta_t)
    print("\rt = {0:12.9f}, L2 error = {1:12.9e}".format(told.Get(), l2error))